In [165]:
from bs4 import BeautifulSoup
import urllib
from profanity import profanity
from random import shuffle

In [156]:
def make_url(search_string):
    search = search_string.replace(" ", "%20")
    full = "https://www.poetryfoundation.org/search?query=" + search + "&refinement=poems"
    return full

In [157]:
def get_links(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    links = soup.find_all("h2", class_="c-hdgSans c-hdgSans_2")
    for i in range(len(links)):
        links[i] = links[i].a["href"]
    return(links)

In [158]:
def get_poem(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    lines = []
    poem = soup.find("div", class_="o-poem")
    if poem:
        lines = poem.find_all("div")
    out = []
    if len(lines) > 300:
        lines = lines[:201]
    for line in lines:
        poem_line = line.get_text().replace(u'\r ', '')
        if all(ord(char) < 128 for char in poem_line) and not profanity.contains_profanity(poem_line):
            out.append(poem_line)
    return out

In [159]:
def get_poems_from_page(url):
    links = get_links(url)
    poems = []
    for link in links:
        poems.append(get_poem(link))
    return poems

temp = get_poems_from_page(make_url("sheep"))

In [179]:
def combine_poems(list1, list2):
    combined = list1 + list2
    shuffle(combined)
    combined = ["\n".join(x) for x in combined]
    combined = "\n".join(combined)
    return combined

In [182]:
a = [['1', '2', '3'], ['a', 'b', 'c', "\n"]]
b = [['4', '5', '6'], ['d', 'e', 'f']]
print combine_poems(a, b)

1
2
3
4
5
6
a
b
c


d
e
f
